#MODEL WITH DROPOUT LAYER

Imports

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time
from collections import OrderedDict

Preparing Data

In [ ]:
# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

Building the model

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(3,18,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c3',nn.Conv2d(18,48,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
            ('s4',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c5',nn.Conv2d(48,360,kernel_size=(5,5))),
            ('relu5',nn.ReLU()) 
 
            ]))
        
        self.fc = nn.Sequential(OrderedDict([
            ('f6',nn.Linear(360,84)),
            ('dropout',nn.Dropout2d()),
            ('relu6',nn.ReLU()),
            ('f7',nn.Linear(84,10)),
            ('sig7',nn.LogSoftmax(dim=-1))
        ]))
        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        x_convnet = self.convnet(x)
        #print(x_convnet.size())
        x_convnet = x_convnet.view((-1,360))
        #print(x_convnet.size())
        x_fc = self.fc(x_convnet)
        out = x_fc
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out

Train function

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        #print(data.shape)
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Test function

In [ ]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Main function

In [1]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = False
    save_model = True

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)
    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..


100%|██████████| 170498071/170498071 [00:03<00:00, 53230516.38it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306597
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.281235
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.222047
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.157662
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.136626
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.061924
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.068780
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.011430
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.053648
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.977734
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.843810
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.807833
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.843959
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.799652
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.903088
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.717841
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.826853
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.840077
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.780014
Train Epoch: 1 [24320/50000 (49%)]	Loss: 1.765638
Train Epoch: 1 